In [1]:
#relevant libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn

In [2]:
#  a loop that iterates over each page and scrapes all contract title urls to a list called 'links'

# declaring lists to store data in
links = []
start_time = time()
req = 0

#For all the 552 pages, 
for page in range(1,553):
    url = "http://tenders.ppa.gov.gh/contracts?page={}".format(page)
    
    #Make a request
    r = requests.get(url)
    
    #Pause the loop
    sleep(randint(0,1))
    
    #Monitor the requests
    req += 1
    elapsed_time = time() - start_time
    print('Req:{}; Frequency: {} req/s'.format(req, req/elapsed_time))
    clear_output(wait = True)
    
    #Throw a warning for non-200 status codes
    if r.status_code != 200:
            warn('Req: {}; Status code: {}'.format(req, r.status_code))
    
    # Break the loop if the number of requests is greater than expected
    if req > 553:
            warn('Number of requests was greater than expected.')
            break
    
    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(r.content,"html.parser")
    
    #Find the contract urls
    page_soup = soup.find('div', class_='content')
    title_url = page_soup.select(".list-wrap .list-title a")
    
    # Scrape the url
    for link in title_url:
        links.append(link.get('href'))    

Req:552; Frequency: 0.8318014141135376 req/s


In [14]:
# access each contract title link, scrape contract details and assign to dataframe 

## declaring lists to store data in
contract_data_gh = pd.DataFrame()
start_time = time()
req = 0

#For all the 552 contract links,
for link in links:
    
    table_header,table_content=[],[]
    
    #Make a request
    response = requests.get(link)
    
    #Pause a loop
    sleep(randint(1,5))
    
    #Monitor the requests
    req += 1
    elapsed_time = time() - start_time
    print('Req:{}; Frequency: {} req/s'.format(req, req/elapsed_time))
    clear_output(wait = True)
    
    #Throw a warning for non-200 status codes
    if response.status_code != 200:
            warn('Req: {}; Status code: {}'.format(req, response.status_code))
    
    # Break the loop if the number of requests is greater than expected
    if req > 5530:
            warn('Number of requests was greater than expected.')
            break
    
    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    
    #Find the data
    link_soup = soup.find('div', class_='content')
    head = link_soup.select("dl dt")
    content = link_soup.select("dl dd")
    
    #Scrape the data and append to a dataframe
    for item in head:
        table_header.append(item.string)
        
    for item in content:
        table_content.append(item.string)

    contracts = pd.DataFrame(dict(zip(table_header, table_content)), index=[0])   

    contract_data_gh=contract_data_gh.append(contracts)

ConnectionError: HTTPConnectionPool(host='tenders.ppa.gov.gh', port=80): Max retries exceeded with url: /contracts/7413 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fbf66796208>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [5]:
#view first five rows of dataframe

contract_data_gh.head()

,Awarding Agency:,Tender Package No:,Tender Type:,Contract Type:,Lot #:,Tender Description:,Approval Auth:,Justification:,Contract Date:,Completion Date:,Contract Currency:,Contract Award Price:,Contract Awarded To:,Company Email:,Company Address:,Company Tel:,Supplier #:
0,Ghana Irrigation Development Authority - Accra,MOFA/GIDA-IFT/LOT 9-SAN-DAG/12/18,-,Restricted Tender,LOT 9,Construction of small earth dams to store wate...,ETC/PPA,40 (1)b - Urgency,"10th May, 2019","20th May, 2020",Ghana Cedi,Gh¢4962525.82,Ashcarl Investment Ltd,None,P. O. Box 2310 Tamale,None,-
0,Ghana Irrigation Development Authority - Accra,MOFA/GIDA-IFT/LOT 1-VUN-NAM/12/18,-,Restricted Tender,LOT 1,Construction of Small earth dams at Vunania an...,ETC/PPA,40 (1)b - Urgency,"10th May, 2019","12th May, 2020",Ghana Cedi,Gh¢5087867.52,ALZAK COMPANY LIMITED,info@alzak.com,-,-,-
0,Ghana Irrigation Development Authority - Accra,MOFA/GIDA-IFT/LOT 3-KATAA/12/18,-,Restricted Tender,LOT 3,Construction of small earth dam at Kataa for a...,ETC/PPA,40 (1)b - Urgency,"10th May, 2019","14th January, 2020",Ghana Cedi,Gh¢2489286.20,QUALITY ASSURED ENGINEERING COMPANY,info@qualityassured.com,-,0,-
0,Ghana Irrigation Development Authority - Accra,MOFA/GIDA-IFT/LOT 4-DUONG/12/18,-,Restricted Tender,LOT 4,Construction of small earth dam at Doung to st...,ETC/PPA,40 (1)b - Urgency,"10th May, 2019","14th January, 2020",Ghana Cedi,Gh¢2576940.88,HANDOSKY INTERNATIONAL LIMITED,support@HANDOSKY.com,-,-,-
0,Ghana Irrigation Development Authority - Accra,MOFA/GIDA-IFT/LOT 5-DOUSE/12/18,-,Restricted Tender,LOT 5,Construction of small earth dams at Douse to s...,ETC/PPA,40 (1)b - Urgency,"10th May, 2019","14th January, 2020",Ghana Cedi,Gh¢2435596.41,WAALE CONSTRUCTION WORKS LIMITED,WAALE@yahoo.com,-,-,-


In [6]:
# Number of rows and columns in dataframe

contract_data_gh.shape

(553, 17)

In [7]:
# setting new index for dataframe and deleting current one

contract_data_gh.reset_index(drop=True, inplace=True)

In [8]:
#converting dataframe to csv

# contract_data_gh.to_csv('contract_data_ppa_gh.csv')